In [1]:
import yank.analyze as yk
import matplotlib as mt
from matplotlib import pyplot as plt
from matplotlib import pylab
from mpl_toolkits.mplot3d import Axes3D
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
from __future__ import print_function
from numpy import dot, sqrt
from numpy.linalg import eigvalsh
#from sklearn.decomposition import PCA
import pandas as pnd
import mdtraj as md
import nglview as nv
import seaborn as sb
import numpy as np
import os as os

In [2]:
# Matplotlib options
%matplotlib inline
mt.style.use('ggplot')
pylab.rcParams['figure.figsize'] = 20, 12

## Set the path for your experiment

Path is the direcrtory where the file analysis.yaml and complex.nc files are located.

In [ ]:
os.chdir ('/DATA/projects/Testing_Yank/binding/t4-lysozyme/p-xylene-explicit-output/experiments')

# Save HDF5 trajectories for each replica

Comment this part of the script once you saved your trajectories if you pretend to restart the kernel. Save trajectory files usually takes several time (depending on the number of the interations of your simulation). Be patient!

In [ ]:
#n_replica= np.arange (25) #Replace this number with the total of replicas you have in your experiment
#for i in n_replica:
    #index=str(i)
    #result= yk.extract_trajectory ('complex.nc',nc_checkpoint_file='complex_checkpoint.nc', replica_index=index, start_frame=0, end_frame=-1, keep_solvent=False)
    #save_file= ('replica_trajectories/'+'replica_'+index+'.h5') #this line needs a replica_trajectories folder in 'experiments' path
    #result.save_hdf5 (save_file) #save in hdf5 format, but it can be replaced by other format depending on your interest, you can check mdtraj manual for more information

## Set the path for replica trajectory files
This folder is the location where you saved the trajectories of the replicas of your simulation.

In [3]:
os.chdir ('/DATA/projects/Testing_Yank/binding/t4-lysozyme/p-xylene-explicit-output/experiments/replica_trajectories')

# Allow to user select a replica to visualize

In [ ]:
n_replica= np.arange (25)
for i in n_replica:
    index=str(input('Insert a replica number, is a int from 0 to 24:'))
    if isinstance (index, str):
            replica=('replica_'+index+'.h5')
            load_replica= md.load_hdf5 (replica)
            print (load_replica, 'number_of_replica:',index)
            break
    else: 
            print ('This is not a valid replica number, try another')
            
view= nv.show_mdtraj (load_replica) #Shows the selected replica
view.clear_representations ()
view.add_ball_and_stick (selection='MOL', color='red', aspectRatio='10')
view.add_cartoon (selection="protein")
view

# Load all replicas as new variables for analysis

In [ ]:
traj={} #dictionary to create each replica as variable
n_replica= np.arange (25)
for i in n_replica:
    index=str(i)
    load=('replica_'+index+'.h5')
    trayectories=traj['{0}'.format(i)]= md.load_hdf5 (load)

In [ ]:
traj

## List of residues of the system

Is not necesary, but it could be useful

In [ ]:
print ('List of residues in this experiment:',[residue for residue in traj['0'].topology.residues])

## Use the replica 0 trajectory to find ligand array

Again, is not necesary, but it could be useful

In [ ]:
ligand=traj['0'].topology.select ('resid 162') # ID: 162 corresponds with ligand numeration in topology of the system
print(ligand) #Shows an array with number of atoms for the ligand

# Define a variable for the ligand trayectory in each replica

In [4]:
p_xylene={} #dictionary to create p-xylene trayectories for each replica
n_replica= np.arange (25)
for i in n_replica:
    index=str(i)
    load=('replica_'+index+'.h5')
    trayectories=p_xylene['{0}'.format(i)]= md.load_hdf5 (load, atom_indices=[2603,2604,2605,2606,2607,2608,2609,2610,2611,2612,2613,2614,2615,2616,2617,2618,2619,2620])

In [ ]:
p_xylene

## Create a clustering plot for two replicas

replica 1 and 25

In [ ]:
distances = np.empty ((p_xylene['0'].n_frames, p_xylene['0'].n_frames))
for i in range(p_xylene['0'].n_frames):
    distances[i] = md.rmsd(p_xylene['0'],p_xylene['0'], i)
print('Max pairwise rmsd: %f nm' % np.max(distances))

In [ ]:
reduced_distances = squareform(distances, checks=False)
linkage = scipy.cluster.hierarchy.linkage(reduced_distances, method='average')

In [ ]:
plt.title('RMSD Average linkage hierarchical clustering')
plot = scipy.cluster.hierarchy.dendrogram(linkage, no_labels=True, count_sort='descendent')

In [ ]:
rmsd=md.rmsd (p_xylene['0'],p_xylene['0'],precentered=True)
plt.plot(rmsd)
plt.xlabel("saved frame")
plt.ylabel("RMSD (A)")
plt.show()

# Calculating gemetric center for p-xylene

Lets compute all pairwise rmsds between conformations.

In [5]:
atom_indices = [a.index for a in p_xylene['0'].topology.atoms if a.element.symbol != 'H']
distances = np.empty((p_xylene['0'].n_frames, p_xylene['0'].n_frames))
for i in range(p_xylene['0'].n_frames):
    distances[i] = md.rmsd(p_xylene['0'], p_xylene['0'], i, atom_indices=atom_indices)

The algorithim we're going to use is relatively simple:

    Compute all of the pairwise RMSDs between the conformations. This is O(N^2), so it's not going to scale extremely well to large datasets.
    Transform these distances into similarity scores. Our similarities will calculated as
    sij=e−β⋅dij/dscale

where sij is the pairwise similarity, dij is the pairwise distance, and dscale is the standard deviation of the values of d
, to make the computation scale invariant.
Then, we define the centroid as

argmaxi∑jsij

Using β=1
, this is implemented with the following code:

In [6]:
beta = 1
index = np.exp(-beta*distances / distances.std()).sum(axis=1).argmax()
print(index)

200


In [10]:
centroid = p_xylene['0'][index]
xyz=centroid.xyz
print (xyz)

[[[3.0336459 3.6410975 3.0772035]
  [3.1104593 3.6125329 2.9663253]
  [2.8394556 3.5439138 2.9688373]
  [2.9181948 3.5151231 2.8581152]
  [2.8962576 3.6101913 3.0805566]
  [3.0544853 3.5521636 2.8507893]
  [2.8113923 3.6451108 3.1973872]
  [3.141011  3.5124547 2.730598 ]
  [3.0753136 3.697058  3.1604273]
  [3.2159078 3.6379318 2.960898 ]
  [2.73582   3.5116675 2.9725528]
  [2.8757265 3.4710147 2.7684221]
  [2.7501514 3.7279155 3.1596117]
  [2.7536438 3.5538888 3.2168224]
  [2.8654428 3.670476  3.2894158]
  [3.1026192 3.5670693 2.6435497]
  [3.2436213 3.5467312 2.748772 ]
  [3.136815  3.4055831 2.7062058]]]


In [14]:
xyz_list = [x[:,0,:] for x in xyz]
xyz_list

[array([3.0336459, 3.6410975, 3.0772035], dtype=float32)]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter (xyz, c='b', marker='o')
ax.set_xlabel ('x axis')
ax.set_ylabel ('y axis')
ax.set_zlabel ('z axis')

In [ ]:
plt.plot(centroid.xyz)
plt.xlabel("saved frame")
plt.ylabel("RMSD (A)")
plt.show()

In [ ]:
fw=p_xylene['0'].xyz
print (fw)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter (fw, c='b', marker='o')
ax.set_xlabel ('x axis')
ax.set_ylabel ('y axis')
ax.set_zlabel ('z axis')